In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\victo\anaconda3\envs\workproject\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
# Initialize the model once outside the loop for efficiency
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\victo\anaconda3\envs\workproject\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
job_embeddings = model.encode("python")

cand_emded = model.encode("javascript")

joker = model.encode("rattlesnake")

In [11]:
print(cosine_similarity(job_embeddings.reshape(1,-1), cand_emded.reshape(1,-1)))
print(cosine_similarity(job_embeddings.reshape(1,-1), joker.reshape(1,-1)))

[[0.55302155]]
[[0.32219467]]


In [25]:
job_embeddings = model.encode("python,c++,julia,oracle")

cand_emded = model.encode("rust,go,sql,php")

joker = model.encode("crocodile,lizard,boa,rattlesnake")

In [26]:
print(cosine_similarity(job_embeddings.reshape(1,-1), cand_emded.reshape(1,-1)))
print(cosine_similarity(job_embeddings.reshape(1,-1), joker.reshape(1,-1)))

[[0.34695765]]
[[0.11074138]]


In [4]:
import gensim

In [5]:
from gensim.models import Word2Vec

# Example corpus for training
corpus = [
    ["Python", "is", "a", "popular", "programming", "language"],
    ["Java", "and", "C++", "are", "also", "programming", "languages"],
    ["Python", "can", "be", "used", "for", "machine", "learning"],
    ["Gensim", "is", "a", "library", "for", "natural", "language", "processing"],
]

# Train a Word2Vec model
model = Word2Vec(sentences=corpus, vector_size=50, window=3, min_count=1, workers=4)

# Find most similar words to "Python"
similar_words = model.wv.most_similar("Python", topn=5)

# Display the similar words
print("Most similar words to 'Python':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")


Most similar words to 'Python':
learning: 0.2707
can: 0.2106
processing: 0.1672
languages: 0.1503
popular: 0.1321


In [ ]:
import gensim.downloader as api

# Download the Google News vectors
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Word 'sus' is in the model.


In [7]:
if 'python' in model:
    similar_words = model.most_similar('python', topn=10)
    for word, similarity in similar_words:
        print(f"{word}: {similarity}")
else:
    print("Word 'python' is not in the vocabulary.")

pythons: 0.6688377857208252
Burmese_python: 0.6680364608764648
snake: 0.6606292724609375
crocodile: 0.6591362953186035
boa_constrictor: 0.6443519592285156
alligator: 0.6421656608581543
reptile: 0.6387744545936584
albino_python: 0.6158879995346069
croc: 0.6083583831787109
lizard: 0.6013416647911072


In [9]:
similar_words = model.most_similar('BERT', topn=10)

In [1]:
for word, similarity in similar_words:
        print(f"{word}: {similarity}")

NameError: name 'similar_words' is not defined

In [ ]:
%%bash
# Download the GloVe embeddings
wget http://nlp.stanford.edu/data/glove.6B.zip

# Extract the GloVe embeddings
unzip glove.6B.zip -d glove.6B

/usr/bin/bash: line 2: wget: command not found


Archive:  glove.6B.zip
  inflating: glove.6B/glove.6B.50d.txt  
  inflating: glove.6B/glove.6B.100d.txt  
  inflating: glove.6B/glove.6B.200d.txt  
  inflating: glove.6B/glove.6B.300d.txt  


In [10]:
import os
from gensim.models.keyedvectors import KeyedVectors

# Convert GloVe format to Gensim format
def load_glove_model(glove_file):
    glove_vectors = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)
    return glove_vectors

# Load the 300-dimensional GloVe vectors
glove_file = os.path.join("glove.6B", "glove.6B.300d.txt")
glove_model = load_glove_model(glove_file)


In [11]:
if 'python' in glove_model:
    similar_words = glove_model.most_similar('python', topn=10)
    print("Words similar to 'python':")
    for word, similarity in similar_words:
        print(f"{word}: {similarity}")
else:
    print("Word 'python' is not in the vocabulary.")


Words similar to 'python':
monty: 0.6837380528450012
perl: 0.519283652305603
cleese: 0.5092198252677917
pythons: 0.5007115006446838
php: 0.4942314028739929
grail: 0.4683017134666443
scripting: 0.46761268377304077
skit: 0.4474538266658783
javascript: 0.4312553107738495
spamalot: 0.43117913603782654


In [58]:
def calc_similarity_sbs(applicant_df, job_df):
    """"Calculate cosine simlarity based on BERT embeddings of skills"""

    def semantic_similarity_sbert_base_v2(job,resume):
        """calculate similarity with SBERT all-mpnet-base-v2"""
        model = SentenceTransformer('all-mpnet-base-v2')
        model.eval()
        #Encoding:
        score = 0
        sen = job+resume
        sen_embeddings = model.encode(sen)
        for i in range(len(job)):
            if job[i] in resume:
                score += 1
            else:
                max_cosine_sim = max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0]) 
                if max_cosine_sim >= 0.4:
                    score += max_cosine_sim
        score = score/len(job)  
        return round(score,3)
    
    columns = ['applicant', 'job_id', 'all-mpnet-base-v2_score']
    matching_dataframe = pd.DataFrame(columns=columns)
    
    for job_index in range(job_df.shape[0]):
        columns = ['applicant', 'job_id', 'all-mpnet-base-v2_score']
        matching_dataframe = pd.DataFrame(columns=columns)
        ranking_dataframe = pd.DataFrame(columns=columns)
        
        matching_data = []
        
        for applicant_id in range(applicant_df.shape[0]):
            matching_dataframe_job = {
                "applicant": applicant_df.iloc[applicant_id, 0],
                "job_id": job_index,
                "all-mpnet-base-v2_score": semantic_similarity_sbert_base_v2(job_df['Skills'][job_index], applicant_df['Skills'][applicant_id])
            }
            matching_data.append(matching_dataframe_job)
        
        matching_dataframe = pd.concat([matching_dataframe, pd.DataFrame(matching_data)], ignore_index=True)
    matching_dataframe['rank'] = matching_dataframe['all-mpnet-base-v2_score'].rank(ascending=False)
    return matching_dataframe

In [48]:
import os
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, CrossEncoder


def calc_cross(applicant_df, job_df, N=3, parallel=False):
    """Calculate cosine similarity based on BERT embeddings of combined skills."""

    # Initialize the model once outside the loop for efficiency
    model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    model.max_seq_length = 30
    model.tokenizer.padding_side="right"

     # Precompute job embeddings
    job_df['Skills_Text'] = job_df['Skills'].apply(lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else '')
    query = job_df['Skills_Text'][0]
    # Precompute applicant embeddings
    applicant_df['Skills_Text'] = applicant_df['Skills'].apply(lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else '')
    applicants = applicant_df['Skills_Text'].tolist()

    ranks = model.rank(
        query,
        applicants,
        batch_size=32,
        num_workers=os.cpu_count() // 2 if parallel else 0,
        show_progress_bar=False
    )

    similarity_df = pd.DataFrame(ranks)
    similarity_df['softmaxed'] = F.softmax(torch.tensor(similarity_df['score']))
    similarity_df = similarity_df.join(applicant_df[["name"]], on="corpus_id")
    
    # similarity_df['interview_status'] = similarity_df.index.apply(lambda x: 'Selected' if x <= N else 'Not Selected')

    return similarity_df


In [104]:
def old_calc_similarity(applicant_df, job_df):
    """"Calculate cosine simlarity based on BERT embeddings of skills"""

    def semantic_similarity_sbert_base_v2(job,resume):
        """calculate similarity with SBERT all-mpnet-base-v2"""
        model = SentenceTransformer('all-mpnet-base-v2')
        model.eval()
        #Encoding:
        score = 0
        sen = job+resume
        sen_embeddings = model.encode(sen)
        for i in range(len(job)):
            if job[i] in resume:
                score += 1
            else:
                max_cosine_sim = max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0]) 
                if max_cosine_sim >= 0.4:
                    score += max_cosine_sim
        score = score/len(job)  
        return round(score,3)
    
    columns = ['applicant', 'job_id', 'all-mpnet-base-v2_score']
    matching_dataframe = pd.DataFrame(columns=columns)
    
    for job_index in range(job_df.shape[0]):
        columns = ['applicant', 'job_id', 'all-mpnet-base-v2_score']
        matching_dataframe = pd.DataFrame(columns=columns)
        ranking_dataframe = pd.DataFrame(columns=columns)
        
        matching_data = []
        
        for applicant_id in range(applicant_df.shape[0]):
            matching_dataframe_job = {
                "applicant": applicant_df.iloc[applicant_id, 0],
                "job_id": job_index,
                "all-mpnet-base-v2_score": semantic_similarity_sbert_base_v2(job_df['Skills'][job_index], applicant_df['Skills'][applicant_id])
            }
            matching_data.append(matching_dataframe_job)
        
        matching_dataframe = pd.concat([matching_dataframe, pd.DataFrame(matching_data)], ignore_index=True)
    matching_dataframe['rank'] = matching_dataframe['all-mpnet-base-v2_score'].rank(ascending=False)
    return matching_dataframe

In [2]:
import os
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder


def calc_similarity(applicant_df, job_df, N=3, parallel=False):
    """Calculate cosine similarity based on BERT embeddings of combined skills."""

    # Initialize the model once outside the loop for efficiency
    model = SentenceTransformer('all-mpnet-base-v2')
    model.max_seq_length = 50
    model.tokenizer.padding_side="right"
    model.eval()

    def add_eos(input_examples):
        input_examples = [input_example + model.tokenizer.eos_token for input_example in input_examples]
        return input_examples

     # Precompute job embeddings
    job_df['Skills_Text'] = job_df['Skills'].apply(add_eos)
    job_df['Skills_Text'] = job_df['Skills_Text'].apply(lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else '')
    job_embeddings = model.encode(
        job_df['Skills_Text'].tolist())
    # Precompute applicant embeddings
    applicant_df['Skills_Text'] = applicant_df['Skills'].apply(add_eos)
    applicant_df['Skills_Text'] = applicant_df['Skills_Text'].apply(lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else '')
    applicant_embeddings = model.encode(
        applicant_df['Skills_Text'].tolist(),
        batch_size=32,
        num_workers=os.cpu_count() // 2 if parallel else 0,
        show_progress_bar=False
    )

    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(job_embeddings, applicant_embeddings)

    # Create a DataFrame from the similarity matrix
    similarity_df = pd.DataFrame(similarity_matrix.T, index=applicant_df['name'], columns=job_df.index)
    similarity_df = similarity_df.reset_index().melt(id_vars='name', var_name='job_id', value_name='similarity_score')
    similarity_df['rank'] = similarity_df.groupby('job_id')['similarity_score'].rank(ascending=False)
    similarity_df['interview_status'] = similarity_df['rank'].apply(lambda x: 'Selected' if x <= N else 'Not Selected')

    return similarity_df, applicant_embeddings, job_embeddings

C:\Users\victo\anaconda3\envs\workproject\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [95]:
df_resumes = main.get_resumes("../test_skills")
df_resumes = main.resume_extraction(df_resumes)
df_resumes

,name,raw,Skills
0,1_Skill_AWS,AWS \n,[aws]
1,2_Skills_AWS_Git,AWS \nGit \n,"[aws, git]"
2,3_Skills_AWS_Git_Python,AWS \nGit \nPython \n,"[aws, git, python]"
3,3_Skills_UNRELATED_AWS_Power BI_Tableau,AWS \nPower BI \nTableau \n,"[aws, business intelligence, tableau]"
4,4_Skills_AWS_Git_Python_Azure,AWS \nGit \nPython \nAzure \n,"[aws, git, python, azure]"


In [96]:
# Create DataFrame for jobs
description_file_path = os.path.join('job_descriptions', 'job4.txt')
with open(description_file_path, 'r') as file:
    job_description = file.read()
df_jobs = pd.DataFrame([job_description], columns=["raw"])
df_jobs = main.job_info_extraction(df_jobs)
df_jobs

,raw,Skills
0,"Senior Data Scientist, WTW, London\n\nFull job...","[operations research, data science, modelling,..."


In [49]:
calc_cross(df_resumes, df_jobs)

,corpus_id,score,softmaxed,name
0,4,-2.569773,0.660022,4_Skills_AWS_Git_Python_Azure
1,3,-3.971752,0.162438,3_Skills_UNRELATED_AWS_Power BI_Tableau
2,2,-4.400466,0.105803,3_Skills_AWS_Git_Python
3,1,-5.070609,0.054133,2_Skills_AWS_Git
4,0,-6.193891,0.017605,1_Skill_AWS


In [41]:
res, app_emb, job_emb = calc_similarity(df_resumes, df_jobs)
res.sort_values("rank")

,name,job_id,similarity_score,rank,interview_status
4,4_Skills_AWS_Git_Python_Azure,0,0.365599,1.0,Selected
3,3_Skills_UNRELATED_AWS_Power BI_Tableau,0,0.351982,2.0,Selected
0,1_Skill_AWS,0,0.331589,3.0,Selected
2,3_Skills_AWS_Git_Python,0,0.316115,4.0,Not Selected
1,2_Skills_AWS_Git,0,0.293126,5.0,Not Selected


In [ ]:
res = calc_similarity(df_resumes, df_jobs)
res.sort_values("rank")

,name,job_id,similarity_score,rank,interview_status
4,4_Skills_AWS_Git_Python_Azure,0,0.337064,1.0,Selected
0,1_Skill_AWS,0,0.333620,2.0,Selected
3,3_Skills_UNRELATED_AWS_Power BI_Tableau,0,0.309230,3.0,Selected
2,3_Skills_AWS_Git_Python,0,0.285010,4.0,Not Selected
1,2_Skills_AWS_Git,0,0.233112,5.0,Not Selected


In [10]:
df_resumes = main.get_resumes("resumes")
df_resumes = main.resume_extraction(df_resumes)
df_resumes

,name,raw,Skills
0,hanna_pedersen,\n \n \n \n \n \n \n \n \nEDUCATION \n \nNova...,"[business, analytics, business administration,..."
1,irene_abbateli,"\n \n \nIRENE ABBATELLI \nRome, Italy \n(+39)...","[business, analytics, data analysis, machine l..."
2,Luca_Oeztekin,"LUCA OEZTEKIN\nLisbon, Portugal & Cologne, Ger...","[analytics, artificial intelligence, modelling..."
3,Tim_Gunkel,TIM GUNKEL \n \n \n \n \n \n \n ...,"[business, analytics, machine learning, busine..."
4,Tim_gunkel2,TIM GUNKEL \n \n \n \n \n \n \n ...,"[business, analytics, machine learning, busine..."
5,victor_bjorsvik,"VICTOR BJORSVIK \n \nLisbon, Portugal | +47 91...","[accounting, big data, communications, busines..."


In [102]:
print("cross encoder:")
calc_cross(df_resumes, df_jobs)

cross encoder:


,corpus_id,score,softmaxed,name
0,1,3.623231,0.332022,CV_1_3
1,2,3.303173,0.241083,CV_1_5
2,3,2.982490,0.174943,CV_1_7
3,4,2.849244,0.153119,CV_1_9
4,0,2.411467,0.098833,CV_1_1


In [101]:
print("bag of skills - with special token:")
res, *rest = calc_similarity(df_resumes, df_jobs); print(res.sort_values("rank"))

bag of skills - with special token:
     name job_id  similarity_score  rank interview_status
1  CV_1_3      0          0.812950   1.0         Selected
2  CV_1_5      0          0.782577   2.0         Selected
0  CV_1_1      0          0.738783   3.0         Selected
3  CV_1_7      0          0.735222   4.0     Not Selected
4  CV_1_9      0          0.722924   5.0     Not Selected


In [106]:
print("bag of skills - no special token:")
res = old_calc_similarity(df_resumes, df_jobs); print(res.sort_values("rank"))

bag of skills - no special token:
  applicant job_id  all-mpnet-base-v2_score  rank
1    CV_1_3      0                    0.784   1.0
2    CV_1_5      0                    0.754   2.0
4    CV_1_9      0                    0.747   3.0
3    CV_1_7      0                    0.736   4.0
0    CV_1_1      0                    0.716   5.0


In [103]:
print("skill-by-skill:")
res = calc_similarity_sbs(df_resumes, df_jobs); print(res.sort_values("rank"))

skill-by-skill:
  applicant job_id  all-mpnet-base-v2_score  rank
1    CV_1_3      0                    0.784   1.0
2    CV_1_5      0                    0.754   2.0
4    CV_1_9      0                    0.747   3.0
3    CV_1_7      0                    0.736   4.0
0    CV_1_1      0                    0.716   5.0


In [ ]:
# Initialize the model once outside the loop for efficiency
model = SentenceTransformer('all-mpnet-base-v2')
model.max_seq_length = 30
model.tokenizer.padding_side="right"

In [ ]:
dir(model.tokenizer)

In [75]:
model.tokenizer.add_tokens("<space>", special_tokens=True)

0

In [11]:
import pandas as pd
import main


# Create DataFrame for jobs
description_file_path = os.path.join('job_descriptions', 'job2.txt')
with open(description_file_path, 'r') as file:
    job_description = file.read()
df_jobs = pd.DataFrame([job_description], columns=["raw"])
df_jobs = main.job_info_extraction(df_jobs)



def add_eos(input_examples):
        input_examples = [input_example + model.tokenizer.eos_token for input_example in input_examples]
        return input_examples

# Initialize the model once outside the loop for efficiency
model = SentenceTransformer('all-mpnet-base-v2')
model.max_seq_length = 30
model.tokenizer.padding_side="right"

# Precompute job embeddings
df_jobs['Skills_Text'] = df_jobs['Skills'].apply(add_eos)
df_jobs['Skills_Text'] = df_jobs['Skills_Text'].apply(lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else '')


df_resumes['Skills_Text'] = df_resumes['Skills'].apply(add_eos)
df_resumes['Skills_Text'] = df_resumes['Skills_Text'].apply(lambda x: ' '.join(sorted(set(x))) if isinstance(x, list) else '')

In [ ]:
df_resumes.iloc[]

,name,raw,Skills,Skills_Text
0,hanna_pedersen,\n \n \n \n \n \n \n \n \nEDUCATION \n \nNova...,"[business, analytics, business administration,...",analytics</s> artificial intelligence</s> busi...
1,irene_abbateli,"\n \n \nIRENE ABBATELLI \nRome, Italy \n(+39)...","[business, analytics, data analysis, machine l...",advertising</s> analytics</s> business adminis...
2,Luca_Oeztekin,"LUCA OEZTEKIN\nLisbon, Portugal & Cologne, Ger...","[analytics, artificial intelligence, modelling...",analytics</s> api</s> artificial intelligence<...
3,Tim_Gunkel,TIM GUNKEL \n \n \n \n \n \n \n ...,"[business, analytics, machine learning, busine...",analytics</s> business administration</s> busi...
4,Tim_gunkel2,TIM GUNKEL \n \n \n \n \n \n \n ...,"[business, analytics, machine learning, busine...",analytics</s> api</s> business administration<...
5,victor_bjorsvik,"VICTOR BJORSVIK \n \nLisbon, Portugal | +47 91...","[accounting, big data, communications, busines...",accounting</s> analytics</s> apache spark</s> ...


**